In [1]:
import json

In [2]:
with open('/Users/qfu88/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [3]:
!pip install tqdm

In [4]:
from tqdm.notebook import tqdm_notebook

In [5]:
# Install tmdbsimple
!pip install tmdbsimple

In [6]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [7]:
#define a function to add the certification with the other key information.
def get_movie_certification(imdb_id):
    movie = tmdb.Movies(imdb_id)
    info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
    return info

In [8]:
#define the write_json function
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [9]:
# confirm the get certification information function works by testing these two movies: tt0848228 and tt0332280
import pandas as pd
test_ids = ["tt0848228", "tt0332280"]
results = []
errors = []
for imdb_id in test_ids:
    
    try:
        movie_info = get_movie_certification(imdb_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([imdb_id, e])
    
pd.DataFrame(results)



,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.705,27536,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.880,9823,PG-13


In [10]:
#designate a folder
import os, time,json
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_rating.csv.gz']

In [11]:
#load in the title basics data
basics = pd.read_csv('Data/title_basics.csv.gz')

In [12]:
# create required lists for the loop
## define a list of the year to extract from the API
YEARS_TO_GET = [2000,2001]

In [13]:
#define an errors list 
errors = [ ]

## Start OUTER Loop

In [14]:

for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
         with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)
      #define the IDs to call
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
        #Check for and remove any previously downloaded Movie id's
## Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
# filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    for movie_id in tqdm_notebook(movie_ids_to_get,
                              desc=f'Movies from {YEAR}',
                              position=0,
                              leave=True):
        try:
        # Retrieve then data for the movie id
            temp = get_movie_certification(imdb_id)  
        # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        except Exception as e:
            print(e)
            errors.append([movie_id, e])
        final_year_df = pd.read_json(JSON_FILE)
        final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/2715 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/2840 [00:00<?, ?it/s]

HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/movie/11036/releases?api_key=cd868cf5ad2adc0c1899fbc73f330625 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001BE316045B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
- Total errors: 1
